In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn==0.23.2
!pip install sklearn_crfsuite
!pip install pythainlp


     |████████████████████████████████| 6.8 MB 16.1 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.2 which is incompatible.
     |████████████████████████████████| 743 kB 14.5 MB/s 
     |████████████████████████████████| 11.5 MB 14.1 MB/s 


In [ ]:
import sklearn_crfsuite
import pandas as pd
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from pythainlp.tag import pos_tag_sents
import pythainlp.util

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/colab_data/augment_DATA_NEW_EACH_LABELS_SMALL.csv")
test = pd.read_csv("/content/drive/MyDrive/LST20_Kobkrit/test_df_clean_fromkrokit.csv")

In [ ]:
train = train.groupby('sentence_id').agg(lambda x : list(x))
test = test.groupby('sentence_id').agg(lambda x : list(x))


In [ ]:
train["POS"] = pos_tag_sents(list(train["words"]))
test["POS"] = pos_tag_sents(list(test["words"]))


In [ ]:
train["newPOS"] = train["POS"].apply(lambda x : [i[1] for i in x])
test["newPOS"] = test["POS"].apply(lambda x : [i[1] for i in x])


In [ ]:
_NER_TAGS = [
        "O",
        "B_BRN",
        "B_DES",
        "B_DTM",
        "B_LOC",
        "B_MEA",
        "B_NUM",
        "B_ORG",
        "B_PER",
        "B_TRM",
        "B_TTL",
        "I_BRN",
        "I_DES",
        "I_DTM",
        "I_LOC",
        "I_MEA",
        "I_NUM",
        "I_ORG",
        "I_PER",
        "I_TRM",
        "I_TTL",
        "E_BRN",
        "E_DES",
        "E_DTM",
        "E_LOC",
        "E_MEA",
        "E_NUM",
        "E_ORG",
        "E_PER",
        "E_TRM",
        "E_TTL",
    ]

In [ ]:
train_list = [ [ [train["words"][i][j] , train["newPOS"][i][j] , train["labels"][i][j] ] for j in range(len(train["words"][i]))]   for i in train.index]
test_list = [ [ [test["words"][i][j] , test["newPOS"][i][j] , test["labels"][i][j]] for j in range(len(test["words"][i]))]   for i in test.index]


In [ ]:
train_list

In [ ]:
%%time
from pythainlp import word_vector
import numpy as np
word2vec = word_vector.get_model()


CPU times: user 2.13 s, sys: 76.9 ms, total: 2.21 s
Wall time: 2.18 s


In [ ]:
pythainlp.util.countthai("วันอาทิตย์ที่ 24 มีนาคม2562", ignore_chars="")

In [ ]:
word2vec.wv.vocab['in'].index

In [ ]:
pythainlp.util.is_native_thai('YH')

In [ ]:
word2vec = word_vector.WordVector().get_model()
def contain_num(s):
    return any(i.isdigit() for i in s)
def contain_dot(s):
    return any(i=="." for i in s)
def contain_NOT_THAI(s):
    return any(not pythainlp.util.isthai(i) and not i.isdigit() for i in s)
def w2v_index(s):
    try:
      idnex = word2vec.wv.vocab[s].index
      return True
    except:
      return False
def contain_dash(s):
    return any(i=="-" for i in s)

def contain_slash(s):
    return any(i=="_" for i in s)

def is_native(s):
    return pythainlp.util.is_native_thai(s)
def word2features(sent, i):
    
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word': word,
        'postag': postag,
        'len':len(word),
        'contain_num':contain_num(word),
        'contain_dot':contain_dot(word),
        'contain_not_thai':contain_NOT_THAI(word),
        'w2v_index':w2v_index(word),
        'contain_dash':contain_dash(word),
        'word[:3]':word[:3],
        'word[-3:]':word[-3:],
        'contain_slash':contain_slash(word),
        'native':is_native(word),
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:postag': postag1,
            '-1:word': word1,
            '-1:len':len(word1),
            '-1:contain_num':contain_num(word1),
            '-1:contain_dot':contain_dot(word1),
            '-1:contain_not_thai':contain_NOT_THAI(word1),
            '-1:w2v_index':w2v_index(word1),
            '-1:contain_dash':contain_dash(word1),
            '-1:word[:3]':word1[:3],
            '-1:word[-3:]':word1[-3:],
            '-1:contain_slash':contain_slash(word1),
            '-1:native':is_native(word1),
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag1,
            '+1:word': word1,
            '+1:len':len(word1),
            '+1:contain_num':contain_num(word1),
            '+1:contain_dot':contain_dot(word1),
            '+1:contain_not_thai':contain_NOT_THAI(word1),
            '+1:w2v_index':w2v_index(word1),
            '+1:contain_dash':contain_dash(word1),
            '+1:word[:3]':word1[:3],
            '+1:word[-3:]':word1[-3:],
            '+1:contain_slash':contain_slash(word1),
            '+1:native':is_native(word1),
        })
    else:
        features['EOS'] = True

    if i<len(sent)-2:
      word1 = sent[i+2][0]
      postag1 = sent[i+2][1]
      features.update({
            '+2:postag': postag1,
            '+2:word': word1,
            '+2:len':len(word1),
            '+2:contain_num':contain_num(word1),
            '+2:contain_dot':contain_dot(word1),
            '+2:contain_not_thai':contain_NOT_THAI(word1),
            '+2:w2v_index':w2v_index(word1),
            '+2:contain_dash':contain_dash(word1),
            '+2:word[:3]':word1[:3],
            '+2:word[-3:]':word1[-3:],
            '+2:contain_slash':contain_slash(word1),
            '+2:native':is_native(word1),            
        })
    if i<len(sent)-3:
      word1 = sent[i+3][0]
      postag1 = sent[i+3][1]
      features.update({
            '+3:postag': postag1,
            '+3:word': word1,
            '+3:len':len(word1),
            '+3:contain_num':contain_num(word1),
            '+3:contain_dot':contain_dot(word1),
            '+3:contain_not_thai':contain_NOT_THAI(word1),
            '+3:w2v_index':w2v_index(word1),
            '+3:contain_dash':contain_dash(word1),
            '+3:word[:3]':word1[:3],
            '+3:word[-3:]':word1[-3:],
            '+3:contain_slash':contain_slash(word1),
            '+3:native':is_native(word1),
        })
    if i>1:
      word1 = sent[i-2][0]
      postag1 = sent[i-2][1]
      features.update({
            '-2:postag': postag1,
            '-2:word': word1,
            '-2:len':len(word1),
            '-2:contain_num':contain_num(word1),
            '-2:contain_dot':contain_dot(word1),
            '-2:contain_not_thai':contain_NOT_THAI(word1),
            '-2:w2v_index':w2v_index(word1),
            '-2:contain_dash':contain_dash(word1),
            '-2:word[:3]':word1[:3],
            '-2:word[-3:]':word1[-3:],
            '-2:contain_slash':contain_slash(word1),
            '-2:native':is_native(word1),
        })
    if i>2:
      word1 = sent[i-3][0]
      postag1 = sent[i-3][1]
      features.update({
            '-3:postag': postag1,
            '-3:word': word1,
            '-3:len':len(word1),
            '-3:contain_num':contain_num(word1),
            '-3:contain_dot':contain_dot(word1),
            '-3:contain_not_thai':contain_NOT_THAI(word1),
            '-3:w2v_index':w2v_index(word1),
            '-3:contain_dash':contain_dash(word1),
            '-3:word[:3]':word1[:3],
            '-3:word[-3:]':word1[-3:],
            '-3:contain_slash':contain_slash(word1),
            '-3:native':is_native(word1),
        }) 


    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
train.to_csv("train.csv",index=False)
test.to_csv("test.csv",index=False)

In [ ]:
train_list

[[['สุรยุทธ์', 'NCMN', 'B_PER'],
  ['ยัน', 'VACT', 'O'],
  ['ปฏิเสธ', 'VACT', 'O'],
  ['ลงนาม', 'NCMN', 'O'],
  ['_', 'NCMN', 'O'],
  ['MOU', 'NCMN', 'O'],
  ['_', 'NCMN', 'O'],
  ['กับ', 'RPRE', 'O'],
  ['อียู', 'NCMN', 'B_ORG'],
  ['ไม่', 'NEG', 'O'],
  ['กระทบ', 'VACT', 'O'],
  ['สัมพันธ์', 'VSTA', 'O']],
 [['1', 'NLBL', 'B_DTM'],
  ['_', 'NCMN', 'I_DTM'],
  ['กันยายน', 'NCMN', 'I_DTM'],
  ['_', 'NCMN', 'I_DTM'],
  ['2550', 'NCNM', 'E_DTM'],
  ['_', 'PUNC', 'O'],
  ['12:21', 'NCNM', 'B_DTM'],
  ['_', 'PUNC', 'I_DTM'],
  ['น.', 'CMTR', 'E_DTM']],
 [['นายก', 'NCMN', 'O'],
  ['ฯ', 'PUNC', 'O'],
  ['_', 'NCMN', 'O'],
  ['ย้ำ', 'VATT', 'O'],
  ['ไม่', 'NEG', 'O'],
  ['ลงนาม', 'VSTA', 'O'],
  ['ใน', 'RPRE', 'O'],
  ['_', 'NCMN', 'O'],
  ['MOU', 'NCMN', 'O'],
  ['_', 'NCMN', 'O'],
  ['ร่วมกับ', 'VACT', 'O'],
  ['อียู', 'NCMN', 'B_ORG'],
  ['กรณีที่', 'JSBR', 'O'],
  ['จะ', 'XVBM', 'O'],
  ['เข้า', 'VACT', 'O'],
  ['มา', 'XVAE', 'O'],
  ['สังเกตการณ์', 'VACT', 'O'],
  ['การ', 'FIXN', 'O'],


In [ ]:
X_train = [sent2features(s) for s in train_list]
y_train = [sent2labels(s) for s in train_list]

X_test = [sent2features(s) for s in test_list]
y_test = [sent2labels(s) for s in test_list]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


KeyboardInterrupt: ignored

In [ ]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 1h 15min 37s, sys: 34.1 s, total: 1h 16min 11s
Wall time: 1h 15min 12s


In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,average='weighted', labels=_NER_TAGS)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1465: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.94937105511981

In [ ]:
import pickle

# save
with open('crfmodel.pkl','wb') as f:
    pickle.dump(crf,f)

In [ ]:
f=open('/content/drive/MyDrive/colab_data/submission NER/ne_test.txt','r')
text = f.readlines()
input_text = []

for i in range(len(text)):
  if text[i] != "\n":
    input_text.append(text[i].replace(" ","_").replace(u"\xa0","_").replace("\n","")) #\xa0 คือ space ใน utf8 
  else:
    input_text.append("_")

input_text = input_text[:len(input_text)-1] # ตัด \n ท้ายไฟล์ทิ้ง

In [ ]:
input_text

['นาง',
 'สุดารัตน์',
 '_',
 'เกยุราพันธ์',
 '_',
 'รมว.',
 'สาธารณสุข',
 '_',
 'แถลง',
 'ถึง',
 'การ',
 'จัด',
 'งาน',
 'มหกรรม',
 'กิน',
 'ไก่',
 'ไทย',
 'ปลอด',
 'ภัย',
 '_',
 '100',
 '%',
 '_',
 'ว่า',
 '_',
 'รัฐบาล',
 'ร่วม',
 'กับ',
 'กทม.',
 'จัด',
 'งาน',
 'ดัง',
 'กล่าว',
 'ขึ้น',
 'ที่',
 'ท้องสนามหลวง',
 'ใน',
 'วัน',
 'เสาร์',
 'ที่',
 '_',
 '7',
 '_',
 'ก.พ.',
 'นี้',
 '_',
 'ตั้งแต่',
 '_',
 '16.00',
 'น.',
 'เป็นต้นไป',
 '_',
 'ส่วน',
 'พ.ต.ท.',
 'ทักษิณ',
 '_',
 'ชินวัตร',
 'รัฐมนตรี',
 '_',
 'จะ',
 'เดินทาง',
 'มา',
 'ถึง',
 'ที่',
 'งาน',
 'ใน',
 'เวลา',
 '_',
 '16.30',
 'น.',
 '_',
 'เพื่อ',
 'เยี่ยมชม',
 'นิทรรศการ',
 '_',
 'และ',
 'ประกอบ',
 'อาหาร',
 'เมนู',
 '_',
 'กระเพรา',
 'ไก่',
 'ไข่ดาว',
 'สุก',
 '_',
 'ไก่',
 'ทอด',
 '_',
 'ด้วย',
 'ตัว',
 'เอง',
 'และ',
 'แจกจ่าย',
 'ให้',
 'กับ',
 'ประชาชน',
 '_',
 'โดย',
 'จะ',
 'มี',
 'นาย',
 'สมัคร',
 '_',
 'สุนทรเวช',
 '_',
 'ผู้',
 'ว่า',
 'ฯ',
 'กทม.',
 '_',
 'และ',
 'ม.ล.',
 'ศิริเฉลิม',
 '_',
 'สวัสดิวัฒน์',
 '_

# Predict

In [ ]:
from pythainlp.tokenize import clause_tokenize

kaggle_clause = clause_tokenize(input_text)


Corpus: lst20-cls
- Downloading: lst20-cls 0.2


100%|██████████| 3738912/3738912 [00:00<00:00, 53915320.31it/s]


In [ ]:
kaggle_pos = [  [ tag[1] for tag in clause] for clause in pos_tag_sents(kaggle_clause)]
kaggle_list = [ [ [kaggle_clause[i][j] , kaggle_pos[i][j]  ] for j in range(len(kaggle_clause[i]))]   for i in range(len(kaggle_clause))]


In [ ]:
kaggle_input = [sent2features(s) for s in kaggle_list]



In [ ]:
kaggle_pred = crf.predict(kaggle_input)
kaggle_pred

In [ ]:
len(kaggle_pred)

3648

In [ ]:
prediction=[]
for clause_pred in kaggle_pred:
  prediction+=clause_pred
len(prediction)

In [ ]:
kaggle_input

In [ ]:
id = [i+1 for i in range(len(prediction))]
submission = pd.DataFrame({"Id":id,"Predicted":prediction})

In [ ]:
submission.Predicted.value_counts()

Series([], Name: Predicted, dtype: int64)

In [ ]:
submission.to_csv("submission_crf_post_4.csv",index=False)

In [ ]:
pos_tag = []
for pos in kaggle_pos:
  pos_tag+=pos
len(pos_tag)

In [ ]:
# for i in range(len(prediction)):
#   if pos_tag[i] == "NTTL" and input_text[i]!="_":
#     prediction[i]="B_TTL"

In [ ]:
# for i in range(1,len(prediction)):

#     if prediction[i-1][0] == "I" and prediction[i][0]=="O":
#       if input_text[i-1] == "_":

#         if (i+2<len(prediction) and prediction[i+2][0] in ["E","I"]) or input_text[i]=="_":
#           prediction[i]="I"+prediction[i-1][1:]
#         else :
#           prediction[i]="E"+prediction[i-1][1:] #can be B

#       else :

#         if i-2>=0 and prediction[i-2][0] == "I":
#           prediction[i-1]="E"+prediction[i-1][1:]
#         else :
#           prediction[i-1]="B"+prediction[i-1][1:]

#     elif prediction[i-1][0] == "O" and prediction[i][0]=="I":
#       if input_text[i] == "_":
#         if (i-2>=0 and prediction[i-2][0] == "I") or input_text[i-1]=="_":
#           prediction[i-1]="I"+prediction[i][1:]
#         else :
#           prediction[i-1]="B"+prediction[i][1:]
#       else:
#         prediction[i]="B"+prediction[i][1:]

#     elif prediction[i-1][0] == "E" and prediction[i][0]=="I":
#       if input_text[i] == "_":
#         if (i-2>=0 and prediction[i-2][0] in ["I","B"]) or input_text[i-1]=="_":
#           prediction[i-1]="I"+prediction[i][1:]
#         else :
#           prediction[i-1]="B"+prediction[i][1:]
#       else:
#         if i-2>=0 and prediction[i-2][0] == "I":
#           prediction[i-1]="I"+prediction[i-1][1:]
#         else :
#           prediction[i-1]="B"+prediction[i-1][1:]
    
#     elif prediction[i-1][0] == "O" and prediction[i][0]=="E":
#       prediction[i]="B"+prediction[i][1:]

#     elif prediction[i-1][0] == "E" and prediction[i][0]=="E":
#       if i-2>=0 and prediction[i-2][0] == "I":
#         prediction[i-1]="I"+prediction[i-1][1:]
#       else :
#         prediction[i]="B"+prediction[i][1:]
    
#     elif prediction[i-1][0]=="I" and prediction[i][0]=="B":
#       if i+1<len(prediction) and prediction[i+1][0]=="E":
#         prediction[i]="I"+prediction[i-1][1:]
#       else:
#         prediction[i]="E"+prediction[i][1:]


In [ ]:
pred = pd.read_csv("/content/submission_pure_crf.csv")["Predicted"]
post_pred = pd.read_csv("/content/submission_crf_post.csv")["Predicted"]

FileNotFoundError: ignored

In [ ]:
y_true = []
y_model = []

for i in range(len(y_test)):
  y_true+=y_test[i]
  y_model+=y_pred[i]

In [ ]:
d_t = pd.DataFrame(y_true)
d_p = pd.DataFrame(y_model)

In [ ]:
d_t[d_t[0] == 'B_BRN']

In [ ]:
test1 =pd.read_csv("/content/drive/MyDrive/LST20_Kobkrit/test_df_clean_fromkrokit.csv")

In [ ]:
test1

In [ ]:
list('ไอพ็อด')

['ไ', 'อ', 'พ', '็', 'อ', 'ด']

In [ ]:
tag = 'E_TRM'
test1[(test1.labels==tag)&(test1.predicted!=tag)]

AttributeError: ignored

In [ ]:
'วัดพระขาว'[:3]

'วัด'

In [ ]:
test1[test1.sentence_id == 3062]

,sentence_id,words,labels,predicted
125161,3062,ถั่วลิสง,O,O
125162,3062,พันธุ์,O,O
125163,3062,_,O,O
125164,3062,มข.,B_TRM,O
125165,3062,60-3,E_TRM,O
...,...,...,...,...
125239,3062,เผยแพร่,O,O
125240,3062,สู่,O,O
125241,3062,เกษตรกร,O,O
125242,3062,แล้ว,O,O


In [ ]:
test1[test1.words == 'โลก']

,sentence_id,words,labels,predicted
126,3,โลก,B_LOC,O
2447,59,โลก,O,O
3633,87,โลก,B_LOC,O
3685,89,โลก,B_LOC,O
3705,90,โลก,B_LOC,O
...,...,...,...,...
141267,3458,โลก,O,B_LOC
141599,3466,โลก,O,B_LOC
142061,3478,โลก,O,O
142087,3478,โลก,O,O


In [ ]:
test1['predicted'] = d_p

In [ ]:
d_p[d_t[0] == 'I_BRN']

NameError: ignored

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true,y_model))

In [ ]:
submisssion =pd.read_csv("/content/submission_pure_crf.csv")
submission["words"]= input_text

FileNotFoundError: ignored

In [ ]:
submission.iloc[20000:20060]

In [ ]:
submission[submission["Predicted"]=="B_DES"]["words"].unique()

In [ ]:
import pickle
name = '/content/drive/MyDrive/colab_data/CRF_3.sav'
pickle.dump(crf,open(name,'wb'))

In [ ]:
import pickle
path = '/content/drive/MyDrive/colab_data/CRF_3.sav'
model = pickle.load(open(path,'rb'))

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(40))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-40:])

Top likely transitions:
I_DTM  -> I_DTM   5.095266
I_PER  -> E_PER   5.056772
I_BRN  -> E_BRN   4.995057
B_BRN  -> I_BRN   4.899693
I_TRM  -> I_TRM   4.864172
I_ORG  -> I_ORG   4.712839
I_TRM  -> E_TRM   4.652709
I_DES  -> E_DES   4.538754
B_TRM  -> I_TRM   4.476283
I_NUM  -> E_NUM   4.411952
I_LOC  -> E_LOC   4.154678
I_BRN  -> I_BRN   4.045681
B_BRN  -> E_BRN   4.020010
I_MEA  -> E_MEA   4.003507
I_TTL  -> E_TTL   3.994588
I_ORG  -> E_ORG   3.907753
I_DES  -> I_DES   3.879621
I_MEA  -> I_MEA   3.878198
I_DTM  -> E_DTM   3.835455
B_LOC  -> E_LOC   3.819217
B_DTM  -> I_DTM   3.749408
I_PER  -> I_PER   3.686952
I_LOC  -> I_LOC   3.570156
B_ORG  -> I_ORG   3.441671
B_TRM  -> E_TRM   3.234060
B_ORG  -> E_ORG   3.204242
B_MEA  -> E_MEA   3.093232
B_DTM  -> E_DTM   2.980947
B_PER  -> I_PER   2.975577
I_NUM  -> I_NUM   2.945910
B_LOC  -> I_LOC   2.739763
O      -> O       2.601977
B_TTL  -> I_TTL   2.575248
B_DES  -> I_DES   2.500358
B_DES  -> E_DES   2.473465
B_PER  -> E_PER   2.416337
O   

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(1000))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-1000:])

In [ ]:
pythainlp.util.dict_trie

AttributeError: ignored

In [ ]:
pythainlp.util.text_to_arabic_digit('สอง')

'2'

In [ ]:
มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าพระนคร

In [ ]:
import eli5
eli5.show_weights(crf, top=10)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


From \ To,B_BRN,B_DES,B_DTM,B_LOC,B_MEA,B_NUM,B_ORG,B_PER,B_TRM,B_TTL,E_BRN,E_DES,E_DTM,E_LOC,E_MEA,E_NUM,E_ORG,E_PER,E_TRM,E_TTL,I_BRN,I_DES,I_DTM,I_LOC,I_MEA,I_NUM,I_ORG,I_PER,I_TRM,I_TTL,O
B_BRN,-1.058,-0.316,-0.113,-1.359,-0.552,-1.212,-0.926,-1.391,-0.528,-0.661,4.486,-0.162,-1.164,-1.23,-1.718,-0.506,-2.033,-0.506,-0.507,0.0,4.766,-0.753,-2.04,-2.071,-2.723,-0.614,-3.032,-2.29,-1.912,0.0,-1.173
B_DES,-0.504,-0.961,-1.223,-0.369,0.016,-1.201,-0.192,2.868,-0.554,0.701,-2.47,2.631,-2.652,-4.137,-2.211,-1.212,-3.453,-1.74,-1.16,-1.875,-1.295,2.503,-3.281,-4.573,-2.827,-1.53,-4.072,-3.535,-2.334,-3.001,-1.469
B_DTM,-1.276,-2.591,-1.005,-3.288,0.262,-2.625,-4.04,-2.922,-1.111,-2.523,-1.867,-2.093,2.502,-3.765,-3.873,-3.451,-4.196,-1.889,-3.284,-0.128,-2.646,-2.784,2.601,-4.042,-5.6,-2.991,-5.109,-2.645,-3.134,-1.187,-2.243
B_LOC,-0.195,-1.456,-0.121,0.931,0.36,0.401,-0.23,-1.108,-0.382,-1.69,-1.687,-2.999,-2.884,3.911,-2.763,-1.966,-2.235,-2.758,-1.853,-0.772,-1.963,-2.959,-4.196,2.875,-4.237,-2.42,-2.867,-4.298,-3.028,-1.033,-0.236
B_MEA,-1.032,-1.903,0.047,-1.658,-2.743,-1.534,-2.001,-2.05,0.0,-3.01,-2.173,-1.499,-4.056,-3.687,3.448,-2.017,-4.081,-1.687,-1.264,-0.294,-2.565,-2.64,-6.026,-5.414,1.599,-5.017,-5.588,-4.26,-2.523,-1.039,0.02
B_NUM,0.398,0.515,-1.809,-1.17,1.276,-0.382,-1.664,-0.506,-0.417,-1.128,-1.976,-1.734,-2.678,-2.507,-5.432,1.405,-2.798,-1.443,-1.512,-0.092,-2.229,-2.218,-5.694,-3.797,-6.199,2.592,-5.384,-2.732,-2.084,-0.432,0.133
B_ORG,-1.5,-1.285,-0.498,0.45,0.308,-0.08,-0.199,-2.329,-0.762,-1.69,-2.556,-1.754,-3.329,-1.422,-3.129,-2.812,3.328,-2.96,-1.976,-1.41,-3.141,-3.715,-4.014,-3.915,-4.239,-1.681,2.97,-4.587,-3.302,-1.437,-0.278
B_PER,-0.847,1.355,-0.123,-0.863,-0.192,0.528,-2.267,-2.315,-0.785,-1.148,-1.62,-1.225,-2.586,-3.333,-2.25,-1.42,-3.065,3.329,-0.974,-0.939,-2.068,-3.104,-3.573,-3.948,-3.623,-1.485,-4.338,3.723,-2.527,-1.652,0.109
B_TRM,-0.761,-0.51,-1.483,-2.725,-0.444,-1.447,-2.348,-2.34,-0.997,-0.161,-1.159,-0.526,-1.63,-1.911,-2.102,-0.605,-2.652,-1.882,3.788,0.0,-1.43,-1.083,-3.385,-2.42,-3.482,-1.62,-3.3,-2.785,3.219,-0.201,-1.818
B_TTL,-0.79,-0.757,-1.457,-2.2,-1.195,-2.847,-3.038,3.247,-0.827,-2.436,-1.503,-2.231,-2.831,-3.514,-2.692,-2.004,-4.485,-2.437,-1.443,2.86,-1.469,-3.632,-2.66,-4.047,-4.577,-1.667,-4.597,-3.143,-2.189,3.134,-2.949


In [ ]:
s